# Bisnis.com

In [85]:
import time
import winsound
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

import re
import warnings
from datetime import datetime, date, timedelta
import pandas as pd
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [86]:
### Initialize Browser (Chrome)
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_extension("uBlock-Origin.crx")
driver = webdriver.Chrome(options=options)

In [87]:
def check_premium(i):
    try:
        pict = driver.find_elements(By.XPATH, '//div[@class="col-sm-4"]')[i]
        pict.find_element(By.CSS_SELECTOR, 'span.label.label-default.label-premium')
        return True
    except NoSuchElementException:
        return False

def check_page_num():
    try:
        driver.find_element(By.XPATH, '//*[@class="pages page-indeks"]')
        return True
    except NoSuchElementException:
        return False

def check_notlive(i):
    try:
        driver.find_elements(By.XPATH, '//div[@class="col-sm-8"]')[i].find_element(By.TAG_NAME, 'span')
        return False
    except NoSuchElementException:
        return True

def news_not_valid(i):
    if driver.find_elements(By.XPATH, '//div[@class="channel"]')[i].find_element(By.TAG_NAME, 'a').text == '':
        return True
    else:
        return False
    
def premium_inside():
    try:
        driver.find_element(By.XPATH, '//*[@class="theme--ekonomi theme--premium -scroll -nav"]')
        return True
    except NoSuchElementException:
        return False

def check_link(url):
    urls = ["koran.bisnis.com", "teknologi.bisnis.com", "lifestyle.bisnis.com", "infografik.bisnis.com", 'traveling.bisnis.com', 'pupr.bisnis.com']
    for i in urls:
        if i in url:
            return True
    return False

In [20]:
news_data = pd.DataFrame(columns=["Tanggal", "Sumber", "Era", "Judul", "Teks"])

In [89]:
page = 643
while True:
    driver.get(f"https://search.bisnis.com/?q=transportasi+umum+jakarta&per_page={page}")
    list_news = driver.find_elements(By.XPATH, '//div[@class="col-sm-8"]')
    count_news = len(list_news)
    if "September 2012" in list_news[0].find_element(By.XPATH, '//div[@class="date"]').text: break
    for curr_news in range(count_news):
        list_news = driver.find_elements(By.XPATH, '//div[@class="col-sm-8"]')
        if check_premium(curr_news): continue
        if not check_notlive(curr_news): continue
        if news_not_valid(curr_news): continue
        curr_link = list_news[curr_news].find_element(By.TAG_NAME, 'a')
        if check_link(curr_link.get_attribute('href')): continue
        curr_link.click()

        driver.switch_to.window(driver.window_handles[1])
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//article[@class="detailsContent force-17 mt40"]')))
        except TimeoutException:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue
        curr_date = driver.find_element(By.XPATH, '//*[@class="detailsAttributeDates"]').text
        title = driver.find_element(By.XPATH, '//*[@class="detailsTitleCaption"]').text
        content = driver.find_element(By.XPATH, '//article[@class="detailsContent force-17 mt40"]')
        curr_text = content.text
        news_data.loc[len(news_data.index)] = [curr_date, "Bisnis.com", "Jokowi", title, curr_text]

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.delete_all_cookies()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//div[@class="col-sm-8"]')))
    page+=1

In [88]:
page

644

In [79]:
driver.switch_to.window(driver.window_handles[0])

In [98]:
news_data[news_data['Tanggal'].str.contains("14 April 2014")]

,Tanggal,Sumber,Era,Judul,Teks
3735,"Senin, 14 April 2014 | 14:20",Bisnis.com,Jokowi,KORUPSI PLTU TARAHAN: Politikus PDI-Perjuangan...,"Bisnis.com, JAKARTA - Majelis Hakim pengadilan..."


In [93]:
news_data.to_csv("TempDataBisnis.csv", index=False)

In [91]:
news_data.drop_duplicates(inplace=True)
news_data.reset_index(drop=True, inplace=True)
news_data

,Tanggal,Sumber,Era,Judul,Teks
0,"Minggu, 13 November 2022 | 17:13",Bisnis.com,Anies,Siap-Siap! Gedung di Jakarta Wajib Pasang Char...,"Bisnis.com, JAKARTA - Kementerian Perhubungan ..."
1,"Selasa, 8 November 2022 | 15:26",Bisnis.com,Anies,"Anggaran Kemacetan Jakarta Capai Rp8,5 Triliun...","Bisnis.com, JAKARTA— Pengamat Transportasi Djo..."
2,"Kamis, 3 November 2022 | 11:00",Bisnis.com,Anies,Jaksa Minta Hakim Tolak Eksepsi Baiquni Wibowo,"Bisnis.com, JAKARTA - Jaksa penuntut umum (JPU..."
3,"Rabu, 2 November 2022 | 15:18",Bisnis.com,Anies,ION Mobility Kerja Sama Dengan PLN Perkuat Jar...,"Bisnis.com, JAKARTA — Produsen motor listrik p..."
4,"Rabu, 26 Oktober 2022 | 19:08",Bisnis.com,Anies,"Menhub Beri Restu, Akuisisi KCI oleh MITJ Masu...","Bisnis.com, JAKARTA - Wacana akuisisi atau mer..."
...,...,...,...,...,...
6315,"Jumat, 28 September 2012 | 16:43",Bisnis.com,Jokowi,"STARBUS JADI ANDALAN, Tata Motors Incar Segmen...","JAKARTA: PT Tata Motors Indonesia (TMI), Agen ..."
6316,"Jumat, 28 September 2012 | 08:26",Bisnis.com,Jokowi,LIGA INGGRIS: Arsenal Umumkan Kenaikan Keuntungan,JAKARTA: Arsenal mengumumkan keuntungan sebesa...
6317,"Jumat, 28 September 2012 | 04:37",Bisnis.com,Jokowi,PON RIAU 2012: DKI serahkan bonus Rp62 miliar ...,JAKARTA - Pemerintah Provinsi DKI Jakarta meny...
6318,"Kamis, 27 September 2012 | 19:35",Bisnis.com,Jokowi,UU PENGADAAN TANAH akan diterapkan secara ketat,JAKARTA: Pemerintah akan memberlakukan Undang-...


# Kompas.com

In [235]:
import time
import winsound
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

import re
import warnings
from datetime import datetime, date, timedelta
import pandas as pd
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [236]:
winsound.PlaySound('*', winsound.SND_ALIAS)
winsound.PlaySound('*', winsound.SND_ALIAS)

In [237]:
def progress_print(count, threshold, maxnum):
    if count == maxnum-1:
        print("======FINISH======")
        winsound.PlaySound('*', winsound.SND_ALIAS)
        winsound.PlaySound('*', winsound.SND_ALIAS)
        winsound.PlaySound('*', winsound.SND_ALIAS)
    elif count % threshold == 0:
        print(f"Progress: {count}/{maxnum}")

In [238]:
### Initialize Browser (Chrome)
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_extension("uBlock-Origin.crx")
driver = webdriver.Chrome(options=options)

In [307]:
driver.get("https://www.google.com/")

In [308]:
driver.find_element(By.ID, 'APjFqb').send_keys('transportasi umum jakarta site:kompas.com after:2017-10-01 before:2022-10-31', Keys.ENTER)

In [309]:
while True:
    try:
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@class="RVQdVd"]'))).click()
    except TimeoutException:
        break

ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF748F82142+3514994]
	(No symbol) [0x00007FF748BA0CE2]
	(No symbol) [0x00007FF748A476AA]
	(No symbol) [0x00007FF748A92E34]
	(No symbol) [0x00007FF748A86A0F]
	(No symbol) [0x00007FF748AB5FEA]
	(No symbol) [0x00007FF748A863B6]
	(No symbol) [0x00007FF748AB6490]
	(No symbol) [0x00007FF748AD28F6]
	(No symbol) [0x00007FF748AB5D93]
	(No symbol) [0x00007FF748A84BDC]
	(No symbol) [0x00007FF748A85C64]
	GetHandleVerifier [0x00007FF748FAE16B+3695259]
	GetHandleVerifier [0x00007FF749006737+4057191]
	GetHandleVerifier [0x00007FF748FFE4E3+4023827]
	GetHandleVerifier [0x00007FF748CD04F9+689705]
	(No symbol) [0x00007FF748BAC048]
	(No symbol) [0x00007FF748BA8044]
	(No symbol) [0x00007FF748BA81C9]
	(No symbol) [0x00007FF748B988C4]
	BaseThreadInitThunk [0x00007FFD959B257D+29]
	RtlUserThreadStart [0x00007FFD9676AA58+40]


In [310]:
news_detect = driver.find_elements(By.XPATH, '//*[@jsname="UWckNb"]')
links_list = [elem.get_attribute('href') for elem in news_detect]
print(len(links_list))

312


In [52]:
#news_data = pd.DataFrame(columns=["Tanggal", "Sumber", "Era", "Judul", "Teks"])

In [311]:
for i in range(len(links_list)):
    status = None
    curr_link = links_list[i]
    if "video.kompas" in curr_link:
        continue
    driver.get(curr_link)
    try:
        WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@class="read__content"]')))
    except TimeoutException:
        if 'image' in curr_link:
            driver.find_element(By.XPATH, '//*[@class="photoArticle__close"]').click()
        elif 'pemilu.kompas.com' in curr_link:
            status = "DifLink"
        else:
            print("Failed to access:", curr_link)
            continue
    
    judul = driver.find_element(By.XPATH, '//*[@class="read__title"]').text
    
    if "amp.kompas.com" in curr_link:
        tanggal = driver.find_element(By.XPATH, '//*[@class="read__date"]').text.split(" ", 3)[2][:-1]
    elif "pemilu.kompas.com" in curr_link:
        tanggal = " ".join(driver.find_element(By.XPATH, '//*[@class="read__time"]').text.split(" ", 4)[1:4])
    else: 
        tanggal = driver.find_element(By.XPATH, '//*[@class="read__time"]').text.split(" ", 3)[2][:-1]
    
    if status == None:
        find_teks = driver.find_element(By.XPATH, '//*[@class="read__content"]').text
        teks = re.split("(\nDapatkan update berita pilihan dan breaking news setiap hari dari Kompas.com.)", find_teks)[0]
    else:
        teks = driver.find_element(By.XPATH, '//*[@class="read__article"]').text
    curr_era = "Anies"

    if teks in news_data['Teks'].to_list():
        matching_indices = news_data[news_data['Teks'] == teks].index
        first_duplicate_index = matching_indices[0]
        if news_data.loc[first_duplicate_index]['Era'] == curr_era:
            continue

    news_data.loc[len(news_data.index)] = [tanggal, "Kompas.com", curr_era, judul, teks]
    progress_print(i, 10, len(links_list))

Failed to access: https://superapps.kompas.com/read/1060735/anies-transportasi-umum-jadi-ruang-kesetaraan-warga-dki-jakarta
Failed to access: https://jeo.kompas.com/dilema-naik-transportasi-umum-di-ibu-kota-bantu-kurangi-macet-tapi-dibikin-ribet
Failed to access: https://jeo.kompas.com/hikayat-becak-di-ibu-kota
Failed to access: https://megapolitan.kompas.com/jeo/dicari-wakil-gubernur-dki-pendamping-anies-baswedan
Progress: 80/312
Failed to access: https://jeo.kompas.com/jalan-panjang-jakarta-jadi-tuan-rumah-formula-e
Progress: 110/312
Failed to access: https://jeo.kompas.com/parahnya-polusi-udara-ibu-kota-dan-bagaimana-jokowi-anies-memperbaikinya
Progress: 140/312
Failed to access: https://www.kompas.com/tag/integrasi
Failed to access: https://tap.kompas.com/read/2020/04/08/06000041/ini-arahan-lengkap-anies-terkait-psbb-jakarta-mulai-jumat-10-april
Progress: 150/312
Progress: 160/312
Progress: 180/312
Progress: 210/312
Failed to access: https://tap.kompas.com/read/2020/04/07/21394831/

In [312]:
i

311

In [313]:
print(len(news_data))
news_data.tail(10)

2308


,Tanggal,Sumber,Era,Judul,Teks
2298,06/09/2022,Kompas.com,Anies,"Tarif Angkot di Sumedang Naik Rp 2.000, Dishub...","SUMEDANG, KOMPAS.com - Tarif angkutan perkotaa..."
2299,13/07/2022,Kompas.com,Anies,"Batik Solo Trans: Harga Tiket, Rute, dan Jam O...",KOMPAS.com - Teman Bus Solo atau Batik Solo Tr...
2300,,Kompas.com,Anies,Anies Baswedan Resmikan Rangkaian MRT,"JAKARTA, KOMPAS.com — Gubernur DKI Jakarta Ani..."
2301,12/09/2022,Kompas.com,Anies,"Resmi, Tarif Angkot di Kabupaten Bogor Naik Rp...","KABUPATEN BOGOR, KOMPAS.com- Pemerintah Kabupa..."
2302,05/02/2021,Kompas.com,Anies,Aplikasi Tije versi Baru Dilengkapi 6 Fitur Ra...,"JAKARTA, KOMPAS.com - PT Transportasi Jakarta ..."
2303,25/10/2022,Kompas.com,Anies,Jenis-Jenis Peta dan Penjelasannya,"Oleh: Yopi Nadia, Guru SDN 106/IX Muaro Sebapo..."
2304,29 Juli 2021,Kompas.com,Anies,"Soal Operasional Transjakarta, Serikat Pekerja...","JAKARTA, KOMPAS.com – Serikat pekerja Tranjaka..."
2305,29/06/2022,Kompas.com,Anies,"Pengaspalan Jalan Cacat Prosedur, Kades dan Ca...","BANGKALAN, KOMPAS.com - Kepala Desa Tanjung Bu..."
2306,16/04/2020,Kompas.com,Anies,Latar Belakang Pendudukan Jepang di Indonesia,KOMPAS.com - Latar belakang pendudukan Jepang ...
2307,Diperbaru,Kompas.com,Anies,Cara Pakai Discord di Smartphone dan Laptop,KOMPAS.com - Pernah mendengar nama Discord? Di...


In [249]:
curr_link = "https://pemilu.kompas.com/read/2022/03/17/09371461/menakar-untung-rugi-tarif-integrasi-transportasi-jak-lingko-yang-akan"
driver.get(curr_link)
judul = driver.find_element(By.XPATH, '//*[@class="read__title"]').text
tanggal = "17/03/2022"
find_teks = driver.find_element(By.XPATH, '//*[@class="read__article"]').text
curr_era = "Anies"

news_data.loc[len(news_data.index)] = [tanggal, "Kompas.com", curr_era, judul, find_teks]

In [232]:
news_data[news_data['Era'] == "Ahok"]

,Tanggal,Sumber,Era,Judul,Teks
782,09/01/2015,Kompas.com,Ahok,Alasan Warga yang Tetap Memilih Naik Kendaraan...,"JAKARTA, KOMPAS.com — Warga Jakarta enggan men..."
783,10/02/2017,Kompas.com,Ahok,Apa Perbedaan JR Connexion dan Transjakarta?,"JAKARTA, KOMPAS.com — Pada 14 Februari 2017 in..."
784,04/01/2016,Kompas.com,Ahok,"Perhatikan, Ini Tarif Sejumlah Bus yang Berope...","JAKARTA, KOMPAS.com — Jalur bus transjakarta a..."
785,26/01/2017,Kompas.com,Ahok,Pencapaian Transjakarta dari Tahun ke Tahun,"JAKARTA, KOMPAS.com - Pada 15 januari 2017, PT..."
786,29/09/2016,Kompas.com,Ahok,Organda DKI: Angkutan Umum Milik Swasta sepert...,"JAKARTA, KOMPAS.com — Anggota DPP Organisasi A..."
...,...,...,...,...,...
1537,16/01/2017,Kompas.com,Ahok,Ketahui Jenis Penyakit Perut Sesuai Titik Saki...,KOMPAS.com - Ada berbagai jenis sakit perut. T...
1538,12/03/2016,Kompas.com,Ahok,Ini Strategi Ahmad Dhani Tingkatkan Elektabilitas,"JAKARTA, KOMPAS.com — Elektabilitas atau tingk..."
1539,15/04/2016,Kompas.com,Ahok,Kaum Profesional yang Tidak Profesional,KOMPAS.com - Apakah Anda tergolong sebagai seo...
1540,26/04/2017,Kompas.com,Ahok,"Sawit, dari Sekadar Tanaman Pagar Menjelma Jad...","PANGKALPINANG, KOMPAS.com - Pada 1968, tanaman..."


In [161]:
news_data.reset_index(inplace=True, drop=True)

In [243]:
#first_news = news_data.copy()

In [208]:
"PT Transjakarta Berencana Pasang Kamera CCTV di Angkot KWK" in news_data["Judul"].to_list()

False

In [314]:
news_data.to_csv("TempDataKompas.csv")

# Detik.com

In [1]:
import time
import winsound
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

import re
import warnings
from datetime import datetime, date, timedelta
import pandas as pd
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
def progress_print(count, threshold, maxnum):
    if count == maxnum-1:
        print("======FINISH======")
        winsound.PlaySound('*', winsound.SND_ALIAS)
    elif count % threshold == 0:
        print(f"Progress: {count}/{maxnum}")

In [3]:
### Initialize Browser (Chrome)
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_extension("uBlock-Origin.crx")
driver = webdriver.Chrome(options=options)

In [4]:
def count_loop():
    news_search = driver.find_element(By.XPATH, '//*[@class="fl text"]').text
    num_news = int(re.findall(r'\d+', news_search)[0])
    num_loop = int(-(-(num_news/9) // 1))
    return num_loop+1

def filter_link():
    extract_potential = driver.find_element(By.XPATH, '//*[@class="list media_rows list-berita"]').find_elements(By.TAG_NAME, 'a')
    link_potential = [ele.get_attribute('href') for ele in extract_potential]
    need_to_pop = []
    for i in range(len(link_potential)):
        if ('/foto-news/' in link_potential[i]) or ('/oto-galeri/' in link_potential[i]) or ('/foto-bisnis/' in link_potential[i]) or ('wolipop' in link_potential[i]):
            need_to_pop.append(i)
    need_to_pop = sorted(need_to_pop, reverse=True)
    for i in need_to_pop:
        link_potential.pop(i)
    return link_potential
    

In [11]:
dnews_data = pd.DataFrame(columns=["Tanggal", "Sumber", "Era", "Judul", "Teks"])

In [13]:
link = "https://www.detik.com/search/searchall?query=transportasi+umum+jakarta&sortby=time&fromdatex=01/10/2012&todatex=31/10/2014"
curr_era = "Jokowi"
driver.get(link)
num_pages = count_loop()
for curr_page in range(4, num_pages):
    try:
        full_link = link+f"&page={curr_page}"
        driver.get(full_link)
    except TimeoutException:
        driver.execute_script("window.stop();")
    news_list = filter_link()

    for curr_news in news_list:
        try:
            driver.get(curr_news)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@class="detail__body-text itp_bodycontent"]')))
            driver.set_page_load_timeout(3)
        except TimeoutException:
            driver.execute_script("window.stop();")
        judul = driver.find_element(By.XPATH, '//*[@class="detail__title"]').text
        if judul == "":
            print("NETNOT")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            judul = driver.find_element(By.XPATH, '//*[@class="detail__title"]').text
            if judul == "": print("AAAAAAAAAAAAAA")
        tanggal = " ".join(driver.find_element(By.XPATH, '//*[@class="detail__date"]').text.split(" ", 4)[1:4])
        if "inet.detik" in curr_news:
            teks = driver.find_element(By.XPATH, '//*[@class="itp_bodycontent detail__body-text"]').text
        else:
            teks = driver.find_element(By.XPATH, '//*[@class="detail__body-text itp_bodycontent"]').text

        dnews_data.loc[len(dnews_data.index)] = [tanggal, "Detik.com", curr_era, judul, teks]
    
    progress_print(curr_page, 10, num_pages)

NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT
NETNOT


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="detail__body-text itp_bodycontent"]"}
  (Session info: chrome=120.0.6099.130); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF685CD2142+3514994]
	(No symbol) [0x00007FF6858F0CE2]
	(No symbol) [0x00007FF6857976AA]
	(No symbol) [0x00007FF6857E1860]
	(No symbol) [0x00007FF6857E197C]
	(No symbol) [0x00007FF685824EE7]
	(No symbol) [0x00007FF68580602F]
	(No symbol) [0x00007FF6858228F6]
	(No symbol) [0x00007FF685805D93]
	(No symbol) [0x00007FF6857D4BDC]
	(No symbol) [0x00007FF6857D5C64]
	GetHandleVerifier [0x00007FF685CFE16B+3695259]
	GetHandleVerifier [0x00007FF685D56737+4057191]
	GetHandleVerifier [0x00007FF685D4E4E3+4023827]
	GetHandleVerifier [0x00007FF685A204F9+689705]
	(No symbol) [0x00007FF6858FC048]
	(No symbol) [0x00007FF6858F8044]

In [15]:
curr_page

9

In [8]:
news_detect = driver.find_elements(By.XPATH, '//*[@jsname="UWckNb"]')
links_list = [elem.get_attribute('href') for elem in news_detect]
print(len(links_list))

0


In [ ]:
dnews_data = pd.DataFrame(columns=["Tanggal", "Sumber", "Era", "Judul", "Teks"])

In [ ]:
i

311

In [ ]:
print(len(news_data))
news_data.tail(10)

2308


,Tanggal,Sumber,Era,Judul,Teks
2298,06/09/2022,Kompas.com,Anies,"Tarif Angkot di Sumedang Naik Rp 2.000, Dishub...","SUMEDANG, KOMPAS.com - Tarif angkutan perkotaa..."
2299,13/07/2022,Kompas.com,Anies,"Batik Solo Trans: Harga Tiket, Rute, dan Jam O...",KOMPAS.com - Teman Bus Solo atau Batik Solo Tr...
2300,,Kompas.com,Anies,Anies Baswedan Resmikan Rangkaian MRT,"JAKARTA, KOMPAS.com — Gubernur DKI Jakarta Ani..."
2301,12/09/2022,Kompas.com,Anies,"Resmi, Tarif Angkot di Kabupaten Bogor Naik Rp...","KABUPATEN BOGOR, KOMPAS.com- Pemerintah Kabupa..."
2302,05/02/2021,Kompas.com,Anies,Aplikasi Tije versi Baru Dilengkapi 6 Fitur Ra...,"JAKARTA, KOMPAS.com - PT Transportasi Jakarta ..."
2303,25/10/2022,Kompas.com,Anies,Jenis-Jenis Peta dan Penjelasannya,"Oleh: Yopi Nadia, Guru SDN 106/IX Muaro Sebapo..."
2304,29 Juli 2021,Kompas.com,Anies,"Soal Operasional Transjakarta, Serikat Pekerja...","JAKARTA, KOMPAS.com – Serikat pekerja Tranjaka..."
2305,29/06/2022,Kompas.com,Anies,"Pengaspalan Jalan Cacat Prosedur, Kades dan Ca...","BANGKALAN, KOMPAS.com - Kepala Desa Tanjung Bu..."
2306,16/04/2020,Kompas.com,Anies,Latar Belakang Pendudukan Jepang di Indonesia,KOMPAS.com - Latar belakang pendudukan Jepang ...
2307,Diperbaru,Kompas.com,Anies,Cara Pakai Discord di Smartphone dan Laptop,KOMPAS.com - Pernah mendengar nama Discord? Di...


In [ ]:
curr_link = "https://pemilu.kompas.com/read/2022/03/17/09371461/menakar-untung-rugi-tarif-integrasi-transportasi-jak-lingko-yang-akan"
driver.get(curr_link)
judul = driver.find_element(By.XPATH, '//*[@class="read__title"]').text
tanggal = "17/03/2022"
find_teks = driver.find_element(By.XPATH, '//*[@class="read__article"]').text
curr_era = "Anies"

news_data.loc[len(news_data.index)] = [tanggal, "Kompas.com", curr_era, judul, find_teks]

In [ ]:
news_data[news_data['Era'] == "Ahok"]

,Tanggal,Sumber,Era,Judul,Teks
782,09/01/2015,Kompas.com,Ahok,Alasan Warga yang Tetap Memilih Naik Kendaraan...,"JAKARTA, KOMPAS.com — Warga Jakarta enggan men..."
783,10/02/2017,Kompas.com,Ahok,Apa Perbedaan JR Connexion dan Transjakarta?,"JAKARTA, KOMPAS.com — Pada 14 Februari 2017 in..."
784,04/01/2016,Kompas.com,Ahok,"Perhatikan, Ini Tarif Sejumlah Bus yang Berope...","JAKARTA, KOMPAS.com — Jalur bus transjakarta a..."
785,26/01/2017,Kompas.com,Ahok,Pencapaian Transjakarta dari Tahun ke Tahun,"JAKARTA, KOMPAS.com - Pada 15 januari 2017, PT..."
786,29/09/2016,Kompas.com,Ahok,Organda DKI: Angkutan Umum Milik Swasta sepert...,"JAKARTA, KOMPAS.com — Anggota DPP Organisasi A..."
...,...,...,...,...,...
1537,16/01/2017,Kompas.com,Ahok,Ketahui Jenis Penyakit Perut Sesuai Titik Saki...,KOMPAS.com - Ada berbagai jenis sakit perut. T...
1538,12/03/2016,Kompas.com,Ahok,Ini Strategi Ahmad Dhani Tingkatkan Elektabilitas,"JAKARTA, KOMPAS.com — Elektabilitas atau tingk..."
1539,15/04/2016,Kompas.com,Ahok,Kaum Profesional yang Tidak Profesional,KOMPAS.com - Apakah Anda tergolong sebagai seo...
1540,26/04/2017,Kompas.com,Ahok,"Sawit, dari Sekadar Tanaman Pagar Menjelma Jad...","PANGKALPINANG, KOMPAS.com - Pada 1968, tanaman..."


In [ ]:
news_data.reset_index(inplace=True, drop=True)

In [ ]:
#first_news = news_data.copy()

In [ ]:
"PT Transjakarta Berencana Pasang Kamera CCTV di Angkot KWK" in news_data["Judul"].to_list()

False

In [ ]:
news_data.to_csv("TempDataKompas.csv")